In [1]:
%pip install rake-nltk skimpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 38.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.1.5
    Uninstalling typeguard-4.1.5:
      Successfully uninstalled typeguard-4.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy

In [2]:
import numpy as np
import os
import pandas as pd
import random
import warnings

from colorama import Fore, Style
from IPython.display import display
from rake_nltk import Rake
from skimpy import skim

warnings.filterwarnings("ignore")

In [3]:
print(f"{Style.BRIGHT}{Fore.BLUE}DIRECTORIES:")
for root, dirs, _ in os.walk("/kaggle/input"):
    for dir in dirs:
        print(f"{Style.BRIGHT}{Fore.GREEN}    {os.path.join(root, dir)}")

print(f"{Style.BRIGHT}{Fore.BLUE}FILES:")
for root, _, files in os.walk("/kaggle/input"):
    for file in files:
        print(f"{Style.BRIGHT}{Fore.GREEN}    {os.path.join(root, file)}")

DIRECTORIES:
    /kaggle/input/nbme-score-clinical-patient-notes
FILES:
    /kaggle/input/nbme-score-clinical-patient-notes/sample_submission.csv
    /kaggle/input/nbme-score-clinical-patient-notes/patient_notes.csv
    /kaggle/input/nbme-score-clinical-patient-notes/features.csv
    /kaggle/input/nbme-score-clinical-patient-notes/train.csv
    /kaggle/input/nbme-score-clinical-patient-notes/test.csv


In [4]:
notes: pd.DataFrame = pd.read_csv("/kaggle/input/nbme-score-clinical-patient-notes/patient_notes.csv")
    
display(notes.describe())
skim(notes)

display(notes.head())

,pn_num,case_num
count,42146.000000,42146.000000
mean,52440.256347,4.943862
std,25165.865233,2.529762
min,0.000000,0.000000
25%,35618.250000,3.000000
50%,50911.500000,5.000000
75%,73037.750000,7.000000
max,95334.000000,9.000000


╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 42146  │ │ int64       │ 2     │                                                          │
│ │ Number of columns │ 3      │ │ string      │ 1     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name     ┃ NA   ┃ NA %   ┃ mean    ┃ sd      ┃ p0  ┃ p25     ┃ p50     ┃ p75     ┃ p100   ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━┩  │
│ │ pn_num          │    0 │      0 │   52440 │   25170 │   0 │   35620 │   50910 │   73040 │  95330 │ ▂▂▇▅▂▆  │  │
│ │ case_num        │    0 │      0 │   4.944 │    2.53 │   0 │       3 │       5 │       7 │      9 │ ▂▁▇▃▃▅  │  │
│ └─────────────────┴──────┴────────┴─────────┴─────────┴─────┴─────────┴─────────┴─────────┴────────┴─────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name               ┃ NA      ┃ NA %       ┃ words per row                ┃ total words              ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ pn_history                │       0 │          0 │                          130 │                  5552255 │  │
│ └───────────────────────────┴─────────┴────────────┴──────────────────────────────┴──────────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...
4,4,0,17yo male with no pmh here for evaluation of p...


In [5]:
# rake: Rake = Rake()
rake: Rake = Rake(punctuations=["(", ")", ",", ".", ":", "),", ")."])

history_id: int = random.randint(0, len(notes) - 1)

rake.extract_keywords_from_text(notes["pn_history"][history_id])

threshold: float = 1.0; score_len: int = -1
print(f"{Style.BRIGHT}{Fore.BLUE}Record ID: {history_id}")
for (score, phrase) in rake.get_ranked_phrases_with_scores():
    if score_len == -1:
        score_len = len("{:.2f}".format(score_len))
    
    if score > threshold:
        print(f"{Style.BRIGHT}{Fore.GREEN}{score:{4 + score_len}.2f}    {phrase}")

Record ID: 16336
    15.00    reports difficulty fallling asleep
     9.00    presenting outpatient clinic
     9.00    per hpi pmh
     9.00    feels overwhelmed constantly
     8.50    significant home stressors
     8.50    father mi social
     7.50    episodes get worse
     5.00    difficulty breathing
     4.50    new stressors
     4.50    etoh social
     4.00    weight changes
     4.00    tylenol allergies
     4.00    particular stressor
     4.00    na psh
     4.00    na meds
     4.00    na fh
     4.00    monday morning
     4.00    gives lectures
     4.00    excessive sweating
     4.00    english professor
     4.00    elederly mother
     4.00    eat something
     4.00    decreased apetite
     4.00    changed roles
     4.00    45yo f
     4.00    - laws
     3.50    sunday night
     3.50    constant nervousness
     1.50    night
     1.50    nervousness
     1.50    episodes
     1.50    episodes
     1.50    episodes
